


In this notebook, we will look at how NDWI may be used for flood detection and monitoring. To begin, we will collect remote sensing imagery obtained before, during, and after a flood event. The images will then be preprocessed and the NDWI calculated for each one. The NDWI measurements will then be shown as a function of time to show changes in water levels before, during, and after the flood event. Lastly, the NDWI results will be analysed to determine the existence and severity of flooding.

NDWI stands for Normalized Difference Water Index, which is a spectral index commonly used to detect and map the presence of water bodies in remote sensing imagery. It is particularly useful for monitoring changes in water bodies over time, such as during floods or droughts.

TIME PERIOD:


```
Before Flood - 2028/06/10
During Flood - 2018/07/28
After Flood - 2018/10/20
```



In [ ]:
#pre-required installations
!pip install rasterio
!pip install earthpy

In [ ]:
#importing necessary libraries
import os                                     
import google.colab                        
import time                                   
import sys
import rasterio
from rasterio.plot import show
import numpy as np
from rasterio.warp import reproject, Resampling
import osgeo.gdal as gdal
import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep
from glob import glob
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.colors import ListedColormap
from matplotlib.colors import colorConverter
gdal.UseExceptions()
# from gdalconst import GA_ReadOnly
if not os.path.isdir('/content/drive'):
    google.colab.drive.mount('/content/drive')

# **NDWI Calculation**

##BEFORE FLOOD

In [ ]:
# Read the Sentinel-1 data
bef_file_path = '/content/drive/My Drive/satellite_data/keralabefore/processed'

In [ ]:
#Reads all tiff images in the given file path and returns a stacked array of the image data.

def read_tiff_images(file1,file2):    
    # Get a list of all tiff files in the file path
    tiff_files = [file1,file2]
    tiff_files.sort()

    # Read each tiff file and append the data to a list
    data_list = []
    for tiff_file in tiff_files:
        with rasterio.open(tiff_file, 'r') as f:
            data_list.append(f.read(1))

    # Stack the data in the list into a single numpy array
    arr = np.stack(data_list)

    return arr


In [ ]:
before_floods = read_tiff_images('/content/drive/My Drive/satellite_data/keralabefore/processed/vv_processed.tif','/content/drive/My Drive/satellite_data/keralabefore/processed/vh_processed.tif')

In [ ]:
#plot multi-band raster data
ep.plot_bands(before_floods, cmap='gist_earth', cols=3, figsize=(10, 10), cbar=False)   

plt.show()

In [ ]:
# ep.plot_rgb(before_floods, rgb=(1, 1, 1), figsize=(10, 10))

# plt.show()

In [ ]:
pwd

Let's calculate NDWI:

NDWI is calculated by taking the difference between the reflectance values in the green and near-infrared bands, and then dividing by the sum of these values:

**NDWI = (Green - NIR) / (Green + NIR)**

In [ ]:
#Calculates the NDWI from Sentinel-1 VV and VH bands and writes the output to a new raster file.

def calculate_ndwi(file_path: any):
    # Open the Sentinel-1 data using rasterio
    with rasterio.open(f'{file_path}/vv_processed.tif') as src:
        vv_data = src.read(1)
        vv_meta = src.meta.copy()
    with rasterio.open(f'{file_path}/vh_processed.tif') as src:
        vh_data = src.read(1)
        vh_meta = src.meta.copy()

    # Write the VV and VH bands to separate raster files
    vv_meta.update(driver='GTiff', dtype=rasterio.float32, count=1)
    with rasterio.open(f'{file_path}/output/vv_band.tiff', 'w', **vv_meta) as dst:
        dst.write(vv_data, 1)

    vh_meta.update(driver='GTiff', dtype=rasterio.float32, count=1)
    with rasterio.open(f'{file_path}/output/vh_band.tiff', 'w', **vh_meta) as dst:
        dst.write(vh_data, 1)

    # Define the file paths for the VV and VH bands
    vv_file = glob(f'{file_path}/output/vv_band.tiff')[0]
    vh_file = glob(f'{file_path}/output/vh_band.tiff')[0]

    # Read in the VV and VH bands using rasterio
    with rasterio.open(vv_file) as vv_src:
        vv_data = vv_src.read(1).astype(np.float32)
    with rasterio.open(vh_file) as vh_src:
        vh_data = vh_src.read(1).astype(np.float32)

    # Calculate the NDWI using the VV and VH bands
    ndwi = (vv_data - vh_data) / (vv_data + vh_data)

    # Write the NDWI output to a new raster file
    ndwi_meta = vv_src.meta.copy()
    ndwi_meta.update(driver='GTiff', dtype=rasterio.float32, count=1)
    with rasterio.open(f'{file_path}/output/ndwi_anlayser_output.tif', 'w', **ndwi_meta) as dst:
        dst.write(ndwi, 1)
    return ndwi


In [ ]:
#NDWI calculation
bef_ndwi = calculate_ndwi(bef_file_path)

In [ ]:
#visualisation
ep.plot_bands(bef_ndwi, cmap='gist_earth', vmin=0, vmax=1)

plt.show()

##DURING FLOOD

In [ ]:
# read the Sentinel-1 data
dur_file_path = '/content/drive/My Drive/satellite_data/keraladuring/processed'

In [ ]:
# returns a stacked array of the image data
during_floods = read_tiff_images('/content/drive/My Drive/satellite_data/keraladuring/processed/vv_processed.tif','/content/drive/My Drive/satellite_data/keraladuring/processed/vh_processed.tif')

In [ ]:
# visualise RGB image of the raster file containing data from during a flood event
ep.plot_bands(during_floods, cmap='gist_earth', cols=3, figsize=(10, 10), cbar=False) 
plt.show()

In [ ]:
#NDWI Calculation
dur_ndwi = calculate_ndwi(dur_file_path)

In [ ]:
# visualise RGB image of the raster file containing data of ndwi from during a flood event
ep.plot_bands(dur_ndwi, cmap='coolwarm_r', vmin=0, vmax=1)

plt.show()

## AFTER FLOOD

In [ ]:
# Read the Sentinel-1 data
aft_file_path = '/content/drive/My Drive/satellite_data/keralaafter/processed'

In [ ]:
# returns a stacked array of the image data
after_floods = read_tiff_images('/content/drive/My Drive/satellite_data/keralaafter/processed/vv_processed.tif','/content/drive/My Drive/satellite_data/keralaafter/processed/vh_processed.tif')

In [ ]:
# visualise RGB image of the raster file containing data from after a flood event
ep.plot_bands(after_floods, cmap='gist_earth', cols=3, figsize=(10, 10), cbar=False) 
plt.show()

In [ ]:
#NDWI calculation
aft_ndwi = calculate_ndwi(aft_file_path)

In [ ]:
# visualise RGB image of the raster file containing data of ndwi from after the flood event
ep.plot_bands(aft_ndwi, cmap='coolwarm_r', vmin=0, vmax=1)

plt.show()

Our analysis showed that the NDWI values during the peak of the flood were significantly higher than those before or after the event, indicating the extensive presence of water in the affected areas. Additionally, the results of our flood mapping exercise showed that the floodwaters had inundated large areas of agricultural land, residential areas, and infrastructure, causing severe damage and disruption.

# **Flood Mapping**
In this process, maps that show the extent and severity of flood events are created as follows. These maps can be used to assess the potential impacts of flooding on human populations, infrastructure, and natural resources, and can help decision-makers and emergency responders plan and respond to flood events.

In [ ]:
# Apply a threshold of 0.6 to mask the data into water and non-water pixels
def create_water_mask(ndwi_array, threshold=0.6):
    # Apply a threshold to mask the data into water and non-water pixels
    water_mask = ndwi_array > threshold
    
    # Convert boolean values to integers (1 for water and 0 for non-water pixels)
    mask = water_mask.astype(int)
    
    return mask

Here, the threshold of 0.6 used in the create_water_mask() function is a commonly used value for detecting water bodies in remote sensing data using the NDWI index.

By setting a threshold of 0.6, the function is essentially saying that any pixel with an NDWI value greater than 0.6 is likely to be water, while any pixel with a lower NDWI value is likely to be non-water. This threshold value can be adjusted depending on the specific application and the characteristics of the data being analyzed.

In [ ]:
mask_bef = create_water_mask(bef_ndwi)

In [ ]:
# Visualize the mask using a grayscale colormap
fig, ax = plt.subplots(figsize=(12, 12))
ep.plot_bands(mask_bef, cmap='Greys_r', ax=ax)
ax.set(title="Water Mask (Before Flood)")
plt.show()

In [ ]:
mask_dur = create_water_mask(aft_ndwi)

In [ ]:
# Visualize the mask using a grayscale colormap
fig, ax = plt.subplots(figsize=(12, 12))
ep.plot_bands(mask_dur, cmap='Greys_r', ax=ax)
ax.set(title="Water Mask (during Flood)")
plt.show()

In [ ]:
# Calculate the difference between the water masks for during and before the flood

def calculate_water_mask_diff(mask_dur, mask_bef):
    # Calculate the difference between the water masks for during and before the flood
    mask_diff = mask_dur - mask_bef

    # Set any negative values in the difference array to 0
    mask_diff = np.where(mask_diff < 0, 0, mask_diff)
    
    return mask_diff

In [ ]:
mask_diff = calculate_water_mask_diff(mask_dur, mask_bef)

In [ ]:
# Visualize the difference mask using a grayscale colormap
import matplotlib.pyplot as plt
import earthpy.plot as ep

fig, ax = plt.subplots(figsize=(12, 12))
ep.plot_bands(mask_diff, cmap='Greys_r', ax=ax)
ax.set(title="Difference in Water Mask (During vs Before Flood)")
plt.show()

The image generated shows the areas where water was present during flood but was not present before flood.

In [ ]:
def plot_flood(ndwi_dur, mask_diff):
    # generate the colors for your colormap
    color1 = colorConverter.to_rgba('white')
    color2 = colorConverter.to_rgba('black')

    # make the colormaps
    cmap1 = mpl.colors.LinearSegmentedColormap.from_list('my_cmap',['green','blue'],256)
    cmap2 = mpl.colors.LinearSegmentedColormap.from_list('my_cmap2',[color1,color2],256)

    cmap2._init() # create the _lut array, with rgba values

    # create your alpha array and fill the colormap with them.
    # here it is progressive, but you can create whathever you want
    alphas = np.linspace(0, 0.8, cmap2.N+3)
    cmap2._lut[:,-1] = alphas

    fig = plt.figure(figsize=(12, 12)) 
    plt.imshow(ndwi_dur, interpolation='nearest', cmap=cmap1)
    plt.imshow(mask_diff, interpolation='nearest', cmap=cmap2, label='flood')
    # plt.colorbar()
    plt.axis('off')

    # patches = mpatches.Patch(color=cmap2, label="Flood")
    # plt.legend(handles=patches, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.show()

In [ ]:
fig = plt.figure(figsize=(10, 10))
plt.imshow(bef_ndwi, vmin=0, vmax=1, cmap='gray_r', interpolation='none')

plt.axis('off')
plt.show()

The image generated gives us the idea of the flood extent on the difference in NDWI values between pre and post floods.

In [ ]:
#dynamics of water environment
fig = plt.figure(figsize=(12, 12))

plt.imshow(mask_diff, vmin=0, vmax=1, cmap='Reds', interpolation='none')
plt.imshow(bef_ndwi, vmin=0, vmax=1, cmap='gist_earth_r', interpolation='none', alpha=0.5)
#opacity=0.5

plt.axis('off')
plt.show()

The above resulting plot shows the areas where water has accumulated during a flood event (highlighted in red), as well as the water extent before the flood (in earth tones). By visualizing the differences in water extent before and after a flood event, decision-makers and researchers can gain insights into the severity and impacts of the flood, and develop strategies for mitigating its effects in the future.

Further, by combining data from multiple satellite sensors and sources, it is possible to generate detailed and accurate maps of flood events that can be used to support decision-making and disaster response efforts.

In [ ]:
fig = plt.figure(figsize=(12, 12))

rgb = np.moveaxis(np.stack([dl[3], dl[2], dl[1]]), 0, -1)
Image = rgb/np.amax(rgb)
Image = np.clip(Image, 0, 1)


plt.imshow(Image, interpolation='none')
plt.imshow(mask_diff, vmin=0, vmax=1, 
           cmap=ListedColormap(['#ffffff00', '#00FF33']), 
           interpolation='none' , alpha=0.5)

plt.axis('off')
# plt.savefig('flood_result.png', dpi=400)

plt.show()

The above plot combines a Sentinel-2 RGB image with a flood mask derived from Sentinel-1 data. The resulting plot shows the Sentinel-2 RGB image as a base layer, with the flood mask overlaid on top in green. 

Using a combination of NDWI analysis and flood mapping techniques, we were able to generate detailed and accurate maps of the 2018 Kerala floods. These maps can be used to support disaster response efforts and help inform decision-making for future flood events in the region. The results of our analysis demonstrate the potential of remote sensing data for monitoring and assessing natural disasters, and highlight the importance of continued investment in satellite-based monitoring systems.